# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
!pip install hyperopt

     |████████████████████████████████| 1.6 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 89.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 70.6 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=ab93264929abeb3d92dad2597e61d8c3bfcbc1c71c476aae7918d93222f1ec70
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/a0/d3/4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built future


In [2]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler 

from sklearn.model_selection import train_test_split


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.impute import SimpleImputer  

from catboost import CatBoostRegressor, Pool, cv
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor


from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


from sklearn.metrics import mean_squared_error


In [3]:
df = pd.read_csv('auto.csv')
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Импортируем данные и выводим на экран.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

В нашем DataFrame 354369 - строк и 15 столбцов. Так же есть значения "NaN". Так же названия столюцов следует привести в приемлемый вид, т е в нижний регистр.

In [5]:
df.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

В 5 столбцах есть значения "Nan". 

In [6]:
df.isnull().sum() * 100 / len(df)

DateCrawled           0.000000
Price                 0.000000
VehicleType          10.579368
RegistrationYear      0.000000
Gearbox               5.596709
Power                 0.000000
Model                 5.560588
Kilometer             0.000000
RegistrationMonth     0.000000
FuelType              9.282697
Brand                 0.000000
NotRepaired          20.079070
DateCreated           0.000000
NumberOfPictures      0.000000
PostalCode            0.000000
LastSeen              0.000000
dtype: float64

Процент значений "NaN" в столбцах. Их значение довольно велико, нужно обработать.

### Изменение регистра столбцов

In [7]:
df.head(2)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50


In [8]:
df = df.rename(columns = 
              {'DateCrawled' : 'date_crawled',
              'Price' : 'price',
              'VehicleType' : 'vehicle_type',
              'RegistrationYear' : 'registration_year',
              'Gearbox' : 'gearbox',
              'Power' : 'power',
              'Model' : 'model',
              'Kilometer' : 'kilometer',
              'RegistrationMonth' : 'registration_month',
              'FuelType' : 'fuel_type',
              'Brand' : 'brand',
              'NotRepaired' : 'not_repaired',
              'DateCreated' : 'date_created',
              'NumberOfPictures' : 'number_of_pictures',
              'PostalCode' : 'postal_code',
              'LastSeen' : 'last_seen'}
             )

In [9]:
df.head(2)

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

Изменили регистр столбцов.

### Удаление дубликатов

In [11]:
df.loc[df.duplicated()]

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
171088,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
231258,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
258109,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
325651,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15


Нашли 4 дубликата, удалим их.

In [12]:
df = df.drop_duplicates()

In [13]:
df.loc[df.duplicated()]

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen


Удалил дубликаты.

### Заполнение в столбцах значений NaN

In [14]:
df.loc[df['not_repaired'].isna()].head(3)

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13


Глядя на Nan в столбце 'not_repaired', я решил заполнить этот столбец исходя из заполнения других ячеек т к если есть данные в других столбцах(в теории машина была на ремонте, т к есть ее данные в других ячейках, потому-что при попадании машины в ремонт, переписывают ее данные + из статей из интернета, можно сделать вывод, что машине с пробегом больше 50,000 км проехать без ремонта практически невозможно, если у нас БД из официальных центрах СТО). Опять же, что мы считаем за ремонт ! Замена колодок, которую рекомендуют делать каждые 10 - 15 тыс км - это по сути тоже ремонт. Поэтому, планку с пробего опускаем до 10,000 км.

In [15]:
df.loc[(df['not_repaired'].isna()) & (df['kilometer'] > 10000), 'not_repaired'] = 'yes'

Заполнили 'not_repaired' исходя из пояснения выше.

Иходя из значение **'NaN'** в остальных столбцах, заполнить их более-менее корректно, не представляется возможным, нет никакой четкой зависимости, например, **volkswagen golf** в типе кузова указаны значения: **small**, **sedan**, **wagon**, **convertible** и др. Для кого-то **volkswagen golf** (как для меня, просмотр первых 20 - 30 авто как 90-х так 2020 годов **volkswagen golf** на Авито - это **"Хетчбэк"**. Для когото - это **"sedan"** или **"small"**.) Не понятно как корректно заполнить пропуски. Так же и для других столбцов. Перебирать каждую машину по типу, модели, виду топлива - не представляется возможным. Чтобы не вносить дополнительные шумы в данные. Принято решение, остальные значения **"NaN"** просто **удалить**. При удалении данных, мы потеряем примерно 20% от изначального их кол-ва. Потери большие, но не критические.

### Удаление выбросов и аномалий DataFrame

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354365 non-null  object
 1   price               354365 non-null  int64 
 2   vehicle_type        316875 non-null  object
 3   registration_year   354365 non-null  int64 
 4   gearbox             334532 non-null  object
 5   power               354365 non-null  int64 
 6   model               334660 non-null  object
 7   kilometer           354365 non-null  int64 
 8   registration_month  354365 non-null  int64 
 9   fuel_type           321470 non-null  object
 10  brand               354365 non-null  object
 11  not_repaired        350666 non-null  object
 12  date_created        354365 non-null  object
 13  number_of_pictures  354365 non-null  int64 
 14  postal_code         354365 non-null  int64 
 15  last_seen           354365 non-null  object
dtypes:

In [17]:
df.describe(include='all')

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
count,354365,354365.000000,316875,354365.000000,334532,354365.000000,334660,354365.000000,354365.000000,321470,354365,350666,354365,354365.0,354365.000000,354365
unique,271174,NaN,8,NaN,2,NaN,250,NaN,NaN,7,40,2,109,NaN,NaN,179150
top,2016-03-24 14:49:47,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,petrol,volkswagen,no,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,NaN,91457,NaN,268249,NaN,29232,NaN,NaN,216349,77012,247158,13718,NaN,NaN,17
mean,NaN,4416.679830,NaN,2004.234481,NaN,110.093816,NaN,128211.363989,5.714650,NaN,NaN,NaN,NaN,0.0,50508.503800,NaN
std,NaN,4514.176349,NaN,90.228466,NaN,189.851330,NaN,37905.083858,3.726432,NaN,NaN,NaN,NaN,0.0,25783.100078,NaN
min,NaN,0.000000,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,1050.000000,NaN,1999.000000,NaN,69.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30165.000000,NaN
50%,NaN,2700.000000,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49413.000000,NaN
75%,NaN,6400.000000,NaN,2008.000000,NaN,143.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71083.000000,NaN


Можно заметить что в столбцах: **"price"**, **"registration_year"** и **"power"** есть нереальные значения **min** и **max**. Будем их обрабатывать. 

In [18]:
df = df.loc[df['price'] > 10]
df = df.loc[(df['registration_year'] > 1950) & (df['registration_year'] < 2020)]
df = df.loc[(df['power'] > 50) & (df['power'] < 500)]

Отфильтровали DataFrame по стоимости машин от 10 долларов, т к год покупки у нас абсолютно любой и состояние машин тоже не известно, поэтому выбрали стоимость в 10 долларов. Год регистрации ограничили с 1950 по 2020, раньше 1950 года машина вряд ли будет стоить так дешево, реаритет всё таки :)) и мощность машины ограничели от 50 до 500 лс, т к если больше 500 лс, то и стоить она будет не 20,000$, а гораздо дороже вне зависимости от года. Считаю, что эти ограничения вполне приемлемы.

In [19]:
df.describe(include='all')

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
count,295833,295833.000000,275716,295833.000000,290411,295833.000000,284337,295833.000000,295833.000000,277448,295833,294901,295833,295833.0,295833.000000,295833
unique,236929,NaN,8,NaN,2,NaN,248,NaN,NaN,7,40,2,106,NaN,NaN,154354
top,2016-03-19 21:49:56,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,petrol,volkswagen,no,2016-04-03 00:00:00,NaN,NaN,2016-04-07 09:44:27
freq,6,NaN,82343,NaN,231185,NaN,25873,NaN,NaN,183339,62766,223472,11693,NaN,NaN,16
mean,NaN,4927.683910,NaN,2003.519935,NaN,122.664753,NaN,128596.911095,5.982967,NaN,NaN,NaN,NaN,0.0,51301.966319,NaN
std,NaN,4601.168568,NaN,6.662681,NaN,52.236656,NaN,36519.610820,3.590904,NaN,NaN,NaN,NaN,0.0,25774.224663,NaN
min,NaN,11.000000,NaN,1951.000000,NaN,51.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,1400.000000,NaN,1999.000000,NaN,82.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30982.000000,NaN
50%,NaN,3300.000000,NaN,2004.000000,NaN,115.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,50321.000000,NaN
75%,NaN,7000.000000,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,72127.000000,NaN


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295833 entries, 1 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        295833 non-null  object
 1   price               295833 non-null  int64 
 2   vehicle_type        275716 non-null  object
 3   registration_year   295833 non-null  int64 
 4   gearbox             290411 non-null  object
 5   power               295833 non-null  int64 
 6   model               284337 non-null  object
 7   kilometer           295833 non-null  int64 
 8   registration_month  295833 non-null  int64 
 9   fuel_type           277448 non-null  object
 10  brand               295833 non-null  object
 11  not_repaired        294901 non-null  object
 12  date_created        295833 non-null  object
 13  number_of_pictures  295833 non-null  int64 
 14  postal_code         295833 non-null  int64 
 15  last_seen           295833 non-null  object
dtypes:

### Копирование DataFrame

In [21]:
df_fill_nan = df.copy()

### Удаление строк со значением NaN

In [22]:
df = df.dropna()

Удалили все строки со значением **NaN**

### Заполнение значений NaN

In [23]:
%time
imp = SimpleImputer(strategy="most_frequent")

df_fill_nan[['fuel_type', 'vehicle_type']] = imp.fit_transform(df_fill_nan[['fuel_type', 'vehicle_type']])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


Для заполнения **NaN** в категорильных столбцах, применили **SimpleImputer** - заполняет наиболее частым значением.

Заполнили столбец **"fuel_type"** значение **"petrol"** т к это самое встречающееся значение столбце.

Заполним столбец **"vehicle_type"** значением **"sedan"** - т к это самое распространенное значение, + **"small"** и **"wagon"** по сравнению с **"sedan"** примерно одинаково стоят, еще следует учитывать тот факт, что **"sedan"**, **"small"** и **"wagon**" разные люди, могут поразному их записать(перепутать, неправильно определить класс машины).

In [24]:
df_fill_nan[df_fill_nan['fuel_type'].isnull()].head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen


Проверяем

In [25]:
df_fill_nan['gearbox'] = df_fill_nan['gearbox'].fillna('Unknown') 
df_fill_nan['model'] = df_fill_nan['model'].fillna('Unknown') 
df_fill_nan['not_repaired'] = df_fill_nan['not_repaired'].fillna('Unknown') 

Заполнил остальные значения **"NaN"** - значением **'Unknown'**. Для того, чтобы сохранить данные, не внося дополнительные шумы.

In [26]:
df_fill_nan.isnull().sum()

date_crawled          0
price                 0
vehicle_type          0
registration_year     0
gearbox               0
power                 0
model                 0
kilometer             0
registration_month    0
fuel_type             0
brand                 0
not_repaired          0
date_created          0
number_of_pictures    0
postal_code           0
last_seen             0
dtype: int64

Все значения **"NaN"** заполнены.

### Удаление ненужных признаков.

In [27]:
colum = ['date_crawled', 
         'registration_month', 
         'date_created', 
         'number_of_pictures', 
         'postal_code', 
         'last_seen']

df = df.drop(columns=colum) 
df_fill_nan = df_fill_nan.drop(columns=colum)
df.head(2)   # Проверяем удаление

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,not_repaired
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,yes
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no


Признак **"registration_year"** - удалять не будем, т к это год покупки машины(нового автомобиля). Оставили только самые важные признаки в DataFrame с удаленными **NaN** значениями и с заполнеными значениями.

### Кодирование категориальных признаков

In [28]:
df.head(2)

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,not_repaired
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,yes
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no


In [29]:
encoder = OrdinalEncoder()

df[['not_repaired', 
    'gearbox', 
    'vehicle_type',
    'fuel_type',
    'model',
    'brand']] = encoder.fit_transform(df[['not_repaired', 
                                          'gearbox', 
                                          'vehicle_type',
                                          'fuel_type',
                                          'model',
                                          'brand']])

df_fill_nan[['not_repaired', 
             'gearbox', 
             'vehicle_type',
             'fuel_type',
             'model',
             'brand']] = encoder.fit_transform(df_fill_nan[['not_repaired', 
                                                            'gearbox', 
                                                            'vehicle_type',
                                                            'fuel_type',
                                                            'model',
                                                            'brand']])

Применяем **OrdinalEncoder** к двум DataFrame т к большое количество категорий в столбцах.

### Разбиение признаков 

In [30]:
target_df = df['price']
features_df = df.drop('price', axis=1)

features_train_df, features_valid_df, target_train_df, target_valid_df = train_test_split(
    features_df.copy(), target_df.copy(), test_size=0.25, random_state=69)


Разбили признаки для DataFrame с удаленными значениями **NaN**

In [31]:
train_fill_nan, validate_fill_nan, test_fill_nan = np.split(df_fill_nan.sample(frac=1, 
                                                                               random_state=42), [int(.6*len(df)), 
                                                                                                  int(.8*len(df))])

features_train_fill_nan = train_fill_nan.drop(['price'], axis=1)
target_train_fill_nan = train_fill_nan.price

features_validate_fill_nan = validate_fill_nan.drop(['price'], axis=1)
target_validate_fill_nan = validate_fill_nan.price

features_test_fill_nan = test_fill_nan.drop(['price'], axis=1)
target_test_fill_nan = test_fill_nan.price


Разбили df на тестовую, валидационную и обучающую выборку.

Сделали ту же операцию и для второго DataFrame 

### Масштабирование количественных признаков

In [32]:
df.head(2)

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,not_repaired
2,9800,6.0,2004,0.0,163,116.0,125000,2.0,14.0,1.0
3,1500,5.0,2001,1.0,75,115.0,150000,6.0,37.0,0.0


In [33]:
numeric = ['vehicle_type', 
           'registration_year', 
           'gearbox',
           'power', 
           'model',                 # Выбираем все столбцы для масштабирования т к 0 и 1 у нас нет в столбцах.
           'kilometer', 
           'fuel_type',
           'brand',
           'not_repaired']    

scaler = StandardScaler()
scaler.fit(features_train_df[numeric])

features_train_df[numeric] = scaler.transform(features_train_df[numeric])
features_valid_df[numeric] = scaler.transform(features_valid_df[numeric]) 


Масштабировали признаки для DataFrame с удаленными значениями **NaN**

In [34]:
numeric = ['vehicle_type', 
           'registration_year', 
           'gearbox',
           'power', 
           'model',           # Выбираем все столбцы для масштабирования т к 0 и 1 у нас нет в столбцах.
           'kilometer', 
           'fuel_type',
           'brand',
           'not_repaired']

scaler = StandardScaler()
scaler.fit(features_train_fill_nan[numeric])

features_train_fill_nan[numeric] = scaler.transform(features_train_fill_nan[numeric])

features_validate_fill_nan[numeric] = scaler.transform(features_validate_fill_nan[numeric]) 

features_test_fill_nan[numeric] = scaler.transform(features_test_fill_nan[numeric]) 

In [35]:
features_test_fill_nan.head(2)

,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,not_repaired
56659,1.335873,-0.976596,0.509176,-0.913240,-0.871984,0.58653,0.674305,0.265593,-0.549889
96324,0.367545,-0.526505,0.509176,-1.199339,0.960680,0.58653,0.674305,1.311455,-0.549889


Сделали ту же операцию и для второго DataFrame. Проверяем результат

**Вывод**


На данном этапе сделали следующее:

- Импортировали DataFrame, посмотрели на данные(типы данных, их кол-во и др),
- Изменили регистр столбцов, првели их в приемлемый вид,
- Удалили дубликаты,
- Заменили значения NaN,
- Удалили NaN,
- Удалили ненужные признаки,
- Закодировали признаки,
- Масштабировали признаки.

## Обучение моделей

### CatBoostRegressor

In [36]:
%%time
param = {'learning_rate': np.arange(0.03, 0.1),
         'depth' : range(1, 5)                           
        }

model = CatBoostRegressor(loss_function = 'RMSE',
                          grow_policy = 'Lossguide',
                          custom_metric = 'RMSE', 
                          eval_metric = 'RMSE',
                          random_state = 69,
                          verbose = 100)


search_random_1 = HalvingRandomSearchCV(model, 
                                      param,      
                                      random_state=69, 
                                      cv=5, 
                                      n_jobs = -1, 
                                      scoring = 'roc_auc').fit(features_train_df, 
                                                               target_train_df, 
                                                               eval_set=(features_valid_df, 
                                                               target_valid_df),
                                                               plot=True)
search_random_1.best_params_

**RMSE = 2226.674608**

In [37]:
search_random_1.best_score_

In [38]:
%%time
param = {'learning_rate': np.arange(0.03, 0.1),
         'depth' : range(1, 5)                           
        }

model = CatBoostRegressor(loss_function = 'RMSE',
                          grow_policy = 'Lossguide',
                          custom_metric = 'RMSE', 
                          eval_metric = 'RMSE',
                          random_state = 69,
                          verbose = 100)


search_random_2 = HalvingRandomSearchCV(model, 
                                      param,      
                                      random_state=69, 
                                      cv=5, 
                                      n_jobs = -1, 
                                      scoring = 'neg_root_mean_squared_error').fit(features_train_fill_nan, 
                                                                                   target_train_fill_nan, 
                                                                                   eval_set=(features_validate_fill_nan, 
                                                                                             target_validate_fill_nan),
                                                                                   plot=True)

search_random_2.best_params_

**RMSE = 1750.479569** Сделали всё тоже самое c CatBoostRegressor, но для второго df с заполнеными значениями NaN

### LightGBM

In [39]:
%%time
model = LGBMRegressor(first_metric_only = True, 
                      metric = 'rmse',
                      random_state = 69)

n_scores = cross_val_score(model, 
                           features_train_df, 
                           target_train_df, 
                           scoring = 'neg_root_mean_squared_error', 
                           cv = 5, 
                           n_jobs = -1)


model.fit(features_train_df, 
          target_train_df,
          eval_set = (features_valid_df, 
                       target_valid_df),
          eval_metric = 'rmse',
          early_stopping_rounds = 10)

**RMSE = 1662.55** 

In [40]:
%%time
model = LGBMRegressor(first_metric_only = True, 
                      metric = 'rmse',
                      random_state = 69)

n_scores = cross_val_score(model, 
                           features_train_df, 
                           target_train_df, 
                           scoring = 'neg_root_mean_squared_error', 
                           cv = 5, 
                           n_jobs = -1)


model.fit(features_train_fill_nan, 
          target_train_fill_nan,
          eval_set = (features_validate_fill_nan, 
                       target_validate_fill_nan),
          eval_metric = 'rmse',
          early_stopping_rounds = 10)

**RMSE = 1736.1** Сделали всё тоже самое c LGBMRegressor, но для второго df с заполнеными значениями NaN

### RandomForestRegressor

In [41]:
%%time
space = {'max_depth': hp.choice('max_depth', np.arange(1, 15, 1, dtype=int)),
         'n_estimators' : hp.choice('n_estimators', np.arange(1, 150, 1, dtype=int))
        }


def objective(space):
    model = RandomForestRegressor(max_depth = space['max_depth'],
                                  n_estimators = space['n_estimators'])
    
    rmse_1 = cross_val_score(model, 
                            features_train_df, 
                            target_train_df, 
                            cv = 5, 
                            scoring = 'neg_root_mean_squared_error', 
                            n_jobs=-1)
     
    return {'loss': abs(rmse_1.mean()), 'status': STATUS_OK }


trials = Trials()

best_1 = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 50,
            trials = trials)

best_1

100%|██████████| 50/50 [48:48<00:00, 58.56s/trial, best loss: 1629.819866537493] 
CPU times: user 48min 38s, sys: 3.59 s, total: 48min 41s
Wall time: 48min 48s


{'max_depth': 13, 'n_estimators': 143}

**RMSE = 1630.05** Лучшие гиперпараметры  **'max_depth': 13, 'n_estimators': 124**

In [42]:
%%time
space = {'max_depth': hp.choice('max_depth', np.arange(1, 15, 1, dtype=int)),
         'n_estimators' : hp.choice('n_estimators', np.arange(1, 150, 1, dtype=int))
        }


def objective(space):
    model = RandomForestRegressor(max_depth = space['max_depth'],
                                  n_estimators = space['n_estimators'])
    
    rmse_2 = cross_val_score(model, 
                            features_train_fill_nan, 
                            target_train_fill_nan, 
                            cv = 5, 
                            scoring = 'neg_root_mean_squared_error', 
                            n_jobs=-1)
     
    return {'loss': abs(rmse_2.mean()), 'status': STATUS_OK }


trials = Trials()

best_2 = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 50,
            trials = trials)

best_2

100%|██████████| 50/50 [37:05<00:00, 44.52s/trial, best loss: 1712.0588994735706]
CPU times: user 36min 59s, sys: 2.06 s, total: 37min 1s
Wall time: 37min 6s


{'max_depth': 13, 'n_estimators': 107}

Сделали всё тоже самое c RandomForestRegressor, но для второго df с заполнеными значениями NaN

## Анализ моделей

Из всех моделей, наилучший результат показала **LightGBM RMSE = 1662.55** Хоть и **RandomForestRegressor** показал немного лучший результат.

In [43]:
%%time
model = LGBMRegressor(first_metric_only = True, 
                      metric = 'rmse',
                      random_state = 69,
                      n_estimators = 100)          # По дефолту 100 итераций, будем использовать как ранний останов.


model.fit(features_train_df, target_train_df) 


pred_valid = model.predict(features_valid_df)
display((mean_squared_error(target_valid_df, pred_valid)) ** 0.5) # считаем rmse

1662.54645544426

CPU times: user 6.29 s, sys: 34.7 ms, total: 6.33 s
Wall time: 6.37 s


Для раннего останова, будем использовать n_estimators, чтобы не делать еще одну выборку для теста.

**RMSE = 1662.54**

## Вывод

**В данном проекте была проведена обработка DataFrame:**
 - Изменение регистра столбцов
 - Удаление дубликатов (хоть у нас их и не много)
 - Заполнение в столбцах значений NaN
 - Удаление выбросов и аномалий в DataFrame
 - Удаление ненужных признаков.
 - Кодирование категориальных признаков
 - Разбиение признаков
 - Масштабирование количественных признаков.

**Обучили модели:**
   - *CatBoostRegressor* с перебором гиперпараметров с помощью HalvingRandomSearchCV с кроссвалидацией 
   - *LightGBM* без перебора гиперпараметров, но с кроссвалидацией
   - *RandomForestRegressor* с перебором гиперпараметров с помощью hyperopt с кроссвалидацией
   
**Время обучения на CatBoostRegressor составило:**
   - На DataFrame с удаленными NaN значениями: процессорное время - 1min 3s,
                                               общее время обучения модели - 1min 38s.
                                               
   - На DataFrame с заполнеными NaN значениями: процессорное время - 1min 39s,
                                                общее время обучения модели - 2min 18s.

**Время обучения на LGBMRegressor составило:**
   - На DataFrame с удаленными NaN значениями: процессорное время - 32min 56s,
                                               общее время обучения модели - 34min 10s.
                                               
   - На DataFrame с заполнеными NaN значениями: процессорное время - 19min 26s,
                                                общее время обучения модели - 19min 49s.
                                                
**Время обучения на RandomForestRegressor составило:**
   - На DataFrame с удаленными NaN значениями: процессорное время - 57min 2s,
                                               общее время обучения модели - 57min 43s.
                                               
   - На DataFrame с заполнеными NaN значениями: процессорное время - 43min 13s,
                                                общее время обучения модели - 43min 41s.                                                

**Результаты метрики RMSE:**
   - *CatBoostRegressor* :
       - На DataFrame с удаленными NaN значениями - 2226.67 на bestIteration = 999.
       - На DataFrame с заполнеными NaN значениями - 1750.47 на bestIteration = 999.

   - *LGBMRegressor* :
       - На DataFrame с удаленными NaN значениями - 1662.55 на bestIteration = 100.
       - На DataFrame с заполнеными NaN значениями - 1736.1 на bestIteration = 100.
       
   - *RandomForestRegressor* :
       - На DataFrame с удаленными NaN значениями - 1630.05 
       - На DataFrame с заполнеными NaN значениями - 1712.6 



Лучшую метрику при обучении показала модель RandomForestRegressor с RMSE = 1630.05 с удаленными значениями NaN, которая обучается без малого 1 час, а это преличное время. Второе место у LGBMRegressor с RMSE = 1662.55 тоже с удаленными значениями NaN, время обучение в 2 раза меньше, чем у RandomForestRegressor и метрики не сильно отличаются.


Из всех результатов и сделанных выводов, выбираем LGBMRegressor с удаленными NaN значениями для итогового результата.

Финальная **RMSE = 1662.54** и общее время обучения модели 1min 27s.